In [2]:
from pandas.io.json import json_normalize
import pandas as pd
import json
import numpy as np
import pandas.io.json as pd_json
from sqlalchemy import create_engine
import matplotlib.pyplot as plt


In [2]:
jsonfile = pd.read_json("Resources/mergedjson.json")
jsonfile

modifiedjson = jsonfile.drop(['target_fg','target_og','contributed_by','boil_volume', 'method', 'volume','attenuation_level'], axis = 1)

In [3]:
#function to scale srm values based on scale
#https://www.twobeerdudes.com/beer/srm
def srmCat(s):
    if s['srm'] >= 1 and s['srm'] < 3:
        return 'Pale Lager, Pilsner, Witbier, Berliner Weisse'
#     elif 3 >= s['srm'] < 4:
    elif s['srm'] >=3 and s['srm'] < 4:
        return 'Straw'
#     elif 4 >= s['srm'] < 6:
    elif s['srm'] >=4 and s['srm'] < 6:
        return 'Pale Gold'
#     elif 6 >= s['srm'] < 8:
    elif s['srm'] >=6 and s['srm'] < 8:
        return 'Deep Gold'
#     elif 8 >= s['srm'] < 10:
    elif s['srm'] >=8 and s['srm'] < 10:
        return 'Pale Amber'
#     elif 10 >= s['srm'] < 13:
    elif s['srm'] >=10 and s['srm'] < 13:
        return 'Medium Amber'
#     elif 13 >= s['srm'] < 17:
    elif s['srm'] >=13 and s['srm'] < 17:
        return 'Deep Amber'
#     elif 17 >= s['srm'] < 20:
    elif s['srm'] >=17 and s['srm'] < 20:
        return 'Amber Brown'
#     elif 20 >= s['srm'] < 24:
    elif s['srm'] >=20 and s['srm'] < 24:
        return 'Brown'
#     elif 24 >= s['srm'] < 35:
    elif s['srm'] >=24 and s['srm'] < 35:
        return 'Ruby Brown'
#     elif 35 >= s['srm'] < 40:
    elif s['srm'] >=35 and s['srm'] < 40:
        return 'Deep Brown'
    elif s['srm'] >= 40:
        return 'Black'
    else:
        return 'Undefined'
    
modifiedjson['srm_category'] = modifiedjson.apply(srmCat, axis=1)

In [4]:
#function to scale ibu into 3 categories 
def ibuCat(i):
    if i['ibu'] >= 0 and i['ibu'] < 25:
        return 'IBU Low'
#   elif 25 >= i['ibu'] < 40:
    elif i['ibu'] >= 25 and i['ibu'] < 40:
        return 'IBU Medium'
    elif i['ibu'] >=40:
        return 'IBU High'
    else:
        return 'IBU Undefined'
    
modifiedjson['ibu_category'] = modifiedjson.apply(ibuCat, axis =1)

In [5]:
#function to turn ibu categories into a number
def ibuCat2(i):
    if i['ibu_category'] == 'IBU Low':
        return 1
#   elif 25 >= i['ibu'] < 40:
    elif i['ibu_category'] == 'IBU Medium':
        return 2
    elif i['ibu_category'] == 'IBU High':
        return 3
    else:
        return 4
    
modifiedjson['ibu_category_nbr'] = modifiedjson.apply(ibuCat2, axis =1)

In [6]:
#function to scale abv into 3 categories 
def abvCat(a):
    if a['abv'] >= 0 and a['abv'] < 9:
        return 'ABV Low'
#   elif 9 >= a['abv'] < 16:
    elif a['abv'] >= 9 and a['abv'] < 16:
        return 'ABV Medium'
    elif a['abv'] >=16:
        return 'ABV High'
    else:
        return 'ABV Undefined'

modifiedjson['abv_category'] = modifiedjson.apply(abvCat, axis =1)    

In [7]:
#function to turn ibu categories into a number
def abvCat2(a):
    if a['abv_category'] == 'ABV Low':
        return 1
    elif a['abv_category'] == 'ABV Medium':
        return 2
    elif a['abv_category'] == 'ABV High':
        return 3
    else:
        return 4

modifiedjson['abv_category_nbr'] = modifiedjson.apply(abvCat2, axis =1)   

In [8]:
beerdata = modifiedjson[['id','name','ibu','ibu_category','ibu_category_nbr','srm','srm_category','abv','abv_category', 'abv_category_nbr', 'ebc','description',
                        'image_url','ingredients','ph','food_pairing', 'tagline','brewers_tips']].set_index('id')
beerdata.to_csv('beerdata.csv')
shortbeerdata = beerdata.drop(columns=['ingredients', 'food_pairing'])
shortbeerdata

,name,ibu,ibu_category,ibu_category_nbr,srm,srm_category,abv,abv_category,abv_category_nbr,ebc,description,image_url,ph,tagline,brewers_tips
id,,,,,,,,,,,,,,,
1,Buzz,60.0,IBU High,3,10.0,Medium Amber,4.50,ABV Low,1,20.0,"A light, crisp and bitter IPA brewed with Engl...",https://images.punkapi.com/v2/keg.png,4.4,A Real Bitter Experience.,The earthy and floral aromas from the hops can...
2,Trashy Blonde,41.5,IBU High,3,15.0,Deep Amber,4.10,ABV Low,1,15.0,"A titillating, neurotic, peroxide punk of a Pa...",https://images.punkapi.com/v2/2.png,4.4,You Know You Shouldn't,Be careful not to collect too much wort from t...
3,Berliner Weisse With Yuzu - B-Sides,8.0,IBU Low,1,4.0,Pale Gold,4.20,ABV Low,1,8.0,Japanese citrus fruit intensifies the sour nat...,https://images.punkapi.com/v2/keg.png,3.2,Japanese Citrus Berliner Weisse.,Clean everything twice. All you want is the cl...
4,Pilsen Lager,55.0,IBU High,3,15.0,Deep Amber,6.30,ABV Low,1,30.0,Our Unleash the Yeast series was an epic exper...,https://images.punkapi.com/v2/4.png,4.4,Unleash the Yeast Series.,Play around with the fermentation temperature ...
5,Avery Brown Dredge,59.0,IBU High,3,5.0,Pale Gold,7.20,ABV Low,1,10.0,An Imperial Pilsner in collaboration with beer...,https://images.punkapi.com/v2/5.png,4.4,Bloggers' Imperial Pilsner.,Make sure you have a big enough yeast starter ...
6,Electric India,38.0,IBU Medium,2,7.5,Deep Gold,5.20,ABV Low,1,15.0,"Re-brewed as a spring seasonal, this beer â€“ ...",https://images.punkapi.com/v2/6.png,4.4,Vibrant Hoppy Saison.,Source some really good heather honey to get t...
7,AB:12,35.0,IBU Medium,2,40.0,Black,11.20,ABV Medium,2,80.0,An Imperial Black Belgian Ale aged in old Inve...,https://images.punkapi.com/v2/7.png,5.3,Imperial Black Belgian Ale.,Don't worry too much about controlling the tem...
8,Fake Lager,40.0,IBU High,3,6.0,Deep Gold,4.70,ABV Low,1,12.0,Fake is the new black. Fake is where it is at....,https://images.punkapi.com/v2/8.png,4.4,Bohemian Pilsner.,Once the primary fermentation is complete get ...
9,AB:07,30.0,IBU Medium,2,42.0,Black,12.50,ABV Medium,2,84.0,Whisky cask-aged imperial scotch ale. Beer per...,https://images.punkapi.com/v2/9.png,5.6,Whisky Cask-Aged Scotch Ale.,Authentic heather honey adds a beautiful flora...


In [9]:
shortbeerdata['ibu_category_nbr'] = shortbeerdata['ibu_category_nbr'].astype(float)
shortbeerdata['abv_category_nbr'] = shortbeerdata['abv_category_nbr'].astype(float)

In [ ]:
# connection_string = "db/beerdata.sqlite"
# engine = create_engine(f'sqlite:///{connection_string}')
# beerdata.to_sql(name='shortbeerdata', con=engine, if_exists='append', index=True)

In [10]:
data_table = beerdata_df.to_html(escape=False)
print(data_table)

#df.to_html(escape=False)

NameError: name 'beerdata_df' is not defined

In [ ]:
binary_beer.to_csv('binary_beer.csv')

In [10]:
# combining columns to get a summary outcome column
data = shortbeerdata
data['outcome'] = data[['ibu_category', 'abv_category', 'srm_category']].apply(lambda x: ' '.join(x), axis=1)
data

,name,ibu,ibu_category,ibu_category_nbr,srm,srm_category,abv,abv_category,abv_category_nbr,ebc,description,image_url,ph,tagline,brewers_tips,outcome
id,,,,,,,,,,,,,,,,
1,Buzz,60.0,IBU High,3.0,10.0,Medium Amber,4.50,ABV Low,1.0,20.0,"A light, crisp and bitter IPA brewed with Engl...",https://images.punkapi.com/v2/keg.png,4.4,A Real Bitter Experience.,The earthy and floral aromas from the hops can...,IBU High ABV Low Medium Amber
2,Trashy Blonde,41.5,IBU High,3.0,15.0,Deep Amber,4.10,ABV Low,1.0,15.0,"A titillating, neurotic, peroxide punk of a Pa...",https://images.punkapi.com/v2/2.png,4.4,You Know You Shouldn't,Be careful not to collect too much wort from t...,IBU High ABV Low Deep Amber
3,Berliner Weisse With Yuzu - B-Sides,8.0,IBU Low,1.0,4.0,Pale Gold,4.20,ABV Low,1.0,8.0,Japanese citrus fruit intensifies the sour nat...,https://images.punkapi.com/v2/keg.png,3.2,Japanese Citrus Berliner Weisse.,Clean everything twice. All you want is the cl...,IBU Low ABV Low Pale Gold
4,Pilsen Lager,55.0,IBU High,3.0,15.0,Deep Amber,6.30,ABV Low,1.0,30.0,Our Unleash the Yeast series was an epic exper...,https://images.punkapi.com/v2/4.png,4.4,Unleash the Yeast Series.,Play around with the fermentation temperature ...,IBU High ABV Low Deep Amber
5,Avery Brown Dredge,59.0,IBU High,3.0,5.0,Pale Gold,7.20,ABV Low,1.0,10.0,An Imperial Pilsner in collaboration with beer...,https://images.punkapi.com/v2/5.png,4.4,Bloggers' Imperial Pilsner.,Make sure you have a big enough yeast starter ...,IBU High ABV Low Pale Gold
6,Electric India,38.0,IBU Medium,2.0,7.5,Deep Gold,5.20,ABV Low,1.0,15.0,"Re-brewed as a spring seasonal, this beer â€“ ...",https://images.punkapi.com/v2/6.png,4.4,Vibrant Hoppy Saison.,Source some really good heather honey to get t...,IBU Medium ABV Low Deep Gold
7,AB:12,35.0,IBU Medium,2.0,40.0,Black,11.20,ABV Medium,2.0,80.0,An Imperial Black Belgian Ale aged in old Inve...,https://images.punkapi.com/v2/7.png,5.3,Imperial Black Belgian Ale.,Don't worry too much about controlling the tem...,IBU Medium ABV Medium Black
8,Fake Lager,40.0,IBU High,3.0,6.0,Deep Gold,4.70,ABV Low,1.0,12.0,Fake is the new black. Fake is where it is at....,https://images.punkapi.com/v2/8.png,4.4,Bohemian Pilsner.,Once the primary fermentation is complete get ...,IBU High ABV Low Deep Gold
9,AB:07,30.0,IBU Medium,2.0,42.0,Black,12.50,ABV Medium,2.0,84.0,Whisky cask-aged imperial scotch ale. Beer per...,https://images.punkapi.com/v2/9.png,5.6,Whisky Cask-Aged Scotch Ale.,Authentic heather honey adds a beautiful flora...,IBU Medium ABV Medium Black


In [28]:
data.to_csv('data.csv')

In [11]:
# working on decision tree (unit 21, 2-4) - i don't think RandomForest will work
from sklearn import tree
import os


In [6]:
# examples from activity - i think we need a new combined column to get to an outcome
target = data2["ibu_category_nbr"]
target_names = [1,2,3,4]

In [8]:
data2 = data2.drop("ibu_category", axis=1)
feature_names = data2.columns
data2.dtypes

id                        int64
name                     object
ibu                     float64
ibu_category_nbr          int64
srm                     float64
srm_category             object
abv                     float64
abv_category             object
abv_category_nbr          int64
ebc                     float64
attenuation_level       float64
attenuation_category     object
atten_category_nbr        int64
description              object
image_url                object
ph                      float64
tagline                  object
brewers_tips             object
malts_name               object
hops_add                 object
hops_attribute           object
hops_name                object
food_pairing             object
dtype: object

In [44]:
data2[data2.name != 'Blitz Series']



,id,name,ibu,ibu_category_nbr,srm,srm_category,abv,abv_category,abv_category_nbr,ebc,description,image_url,ph,tagline,brewers_tips,outcome
0,1,Buzz,60.0,3,10.0,Medium Amber,4.50,ABV Low,1,20.0,"A light, crisp and bitter IPA brewed with Engl...",https://images.punkapi.com/v2/keg.png,4.4,A Real Bitter Experience.,The earthy and floral aromas from the hops can...,IBU High ABV Low Medium Amber
1,2,Trashy Blonde,41.5,3,15.0,Deep Amber,4.10,ABV Low,1,15.0,"A titillating, neurotic, peroxide punk of a Pa...",https://images.punkapi.com/v2/2.png,4.4,You Know You Shouldn't,Be careful not to collect too much wort from t...,IBU High ABV Low Deep Amber
2,3,Berliner Weisse With Yuzu - B-Sides,8.0,1,4.0,Pale Gold,4.20,ABV Low,1,8.0,Japanese citrus fruit intensifies the sour nat...,https://images.punkapi.com/v2/keg.png,3.2,Japanese Citrus Berliner Weisse.,Clean everything twice. All you want is the cl...,IBU Low ABV Low Pale Gold
3,4,Pilsen Lager,55.0,3,15.0,Deep Amber,6.30,ABV Low,1,30.0,Our Unleash the Yeast series was an epic exper...,https://images.punkapi.com/v2/4.png,4.4,Unleash the Yeast Series.,Play around with the fermentation temperature ...,IBU High ABV Low Deep Amber
4,5,Avery Brown Dredge,59.0,3,5.0,Pale Gold,7.20,ABV Low,1,10.0,An Imperial Pilsner in collaboration with beer...,https://images.punkapi.com/v2/5.png,4.4,Bloggers' Imperial Pilsner.,Make sure you have a big enough yeast starter ...,IBU High ABV Low Pale Gold
5,6,Electric India,38.0,2,7.5,Deep Gold,5.20,ABV Low,1,15.0,"Re-brewed as a spring seasonal, this beer â€“ ...",https://images.punkapi.com/v2/6.png,4.4,Vibrant Hoppy Saison.,Source some really good heather honey to get t...,IBU Medium ABV Low Deep Gold
6,7,AB:12,35.0,2,40.0,Black,11.20,ABV Medium,2,80.0,An Imperial Black Belgian Ale aged in old Inve...,https://images.punkapi.com/v2/7.png,5.3,Imperial Black Belgian Ale.,Don't worry too much about controlling the tem...,IBU Medium ABV Medium Black
7,8,Fake Lager,40.0,3,6.0,Deep Gold,4.70,ABV Low,1,12.0,Fake is the new black. Fake is where it is at....,https://images.punkapi.com/v2/8.png,4.4,Bohemian Pilsner.,Once the primary fermentation is complete get ...,IBU High ABV Low Deep Gold
8,9,AB:07,30.0,2,42.0,Black,12.50,ABV Medium,2,84.0,Whisky cask-aged imperial scotch ale. Beer per...,https://images.punkapi.com/v2/9.png,5.6,Whisky Cask-Aged Scotch Ale.,Authentic heather honey adds a beautiful flora...,IBU Medium ABV Medium Black
9,10,Bramling X,75.0,3,11.0,Medium Amber,7.50,ABV Low,1,22.0,"Good old Bramling Cross is elegant, refined, a...",https://images.punkapi.com/v2/10.png,4.4,Single Hop IPA Series - 2011.,Getting good Bramling Cross is key. Most Engli...,IBU High ABV Low Medium Amber


In [45]:
data2[data2.name != 'Ship Wreck']

,id,name,ibu,ibu_category_nbr,srm,srm_category,abv,abv_category,abv_category_nbr,ebc,description,image_url,ph,tagline,brewers_tips,outcome
0,1,Buzz,60.0,3,10.0,Medium Amber,4.50,ABV Low,1,20.0,"A light, crisp and bitter IPA brewed with Engl...",https://images.punkapi.com/v2/keg.png,4.4,A Real Bitter Experience.,The earthy and floral aromas from the hops can...,IBU High ABV Low Medium Amber
1,2,Trashy Blonde,41.5,3,15.0,Deep Amber,4.10,ABV Low,1,15.0,"A titillating, neurotic, peroxide punk of a Pa...",https://images.punkapi.com/v2/2.png,4.4,You Know You Shouldn't,Be careful not to collect too much wort from t...,IBU High ABV Low Deep Amber
2,3,Berliner Weisse With Yuzu - B-Sides,8.0,1,4.0,Pale Gold,4.20,ABV Low,1,8.0,Japanese citrus fruit intensifies the sour nat...,https://images.punkapi.com/v2/keg.png,3.2,Japanese Citrus Berliner Weisse.,Clean everything twice. All you want is the cl...,IBU Low ABV Low Pale Gold
3,4,Pilsen Lager,55.0,3,15.0,Deep Amber,6.30,ABV Low,1,30.0,Our Unleash the Yeast series was an epic exper...,https://images.punkapi.com/v2/4.png,4.4,Unleash the Yeast Series.,Play around with the fermentation temperature ...,IBU High ABV Low Deep Amber
4,5,Avery Brown Dredge,59.0,3,5.0,Pale Gold,7.20,ABV Low,1,10.0,An Imperial Pilsner in collaboration with beer...,https://images.punkapi.com/v2/5.png,4.4,Bloggers' Imperial Pilsner.,Make sure you have a big enough yeast starter ...,IBU High ABV Low Pale Gold
5,6,Electric India,38.0,2,7.5,Deep Gold,5.20,ABV Low,1,15.0,"Re-brewed as a spring seasonal, this beer â€“ ...",https://images.punkapi.com/v2/6.png,4.4,Vibrant Hoppy Saison.,Source some really good heather honey to get t...,IBU Medium ABV Low Deep Gold
6,7,AB:12,35.0,2,40.0,Black,11.20,ABV Medium,2,80.0,An Imperial Black Belgian Ale aged in old Inve...,https://images.punkapi.com/v2/7.png,5.3,Imperial Black Belgian Ale.,Don't worry too much about controlling the tem...,IBU Medium ABV Medium Black
7,8,Fake Lager,40.0,3,6.0,Deep Gold,4.70,ABV Low,1,12.0,Fake is the new black. Fake is where it is at....,https://images.punkapi.com/v2/8.png,4.4,Bohemian Pilsner.,Once the primary fermentation is complete get ...,IBU High ABV Low Deep Gold
8,9,AB:07,30.0,2,42.0,Black,12.50,ABV Medium,2,84.0,Whisky cask-aged imperial scotch ale. Beer per...,https://images.punkapi.com/v2/9.png,5.6,Whisky Cask-Aged Scotch Ale.,Authentic heather honey adds a beautiful flora...,IBU Medium ABV Medium Black
9,10,Bramling X,75.0,3,11.0,Medium Amber,7.50,ABV Low,1,22.0,"Good old Bramling Cross is elegant, refined, a...",https://images.punkapi.com/v2/10.png,4.4,Single Hop IPA Series - 2011.,Getting good Bramling Cross is key. Most Engli...,IBU High ABV Low Medium Amber


In [46]:
data2.dtypes

id                    int64
name                 object
ibu                 float64
ibu_category_nbr      int64
srm                 float64
srm_category         object
abv                 float64
abv_category         object
abv_category_nbr      int64
ebc                 float64
description          object
image_url            object
ph                  float64
tagline              object
brewers_tips         object
outcome              object
dtype: object

In [3]:
# changing all lines in the csv to the same line
data2 = pd.read_csv('finalbeerdata.csv')

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data2, target, random_state=42)

In [12]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

ValueError: could not convert string to float: 'Blitz Series'